In [ ]:
#################### LOOP   window = 18   ########################

# Attention-BLSTM classification of VS+VM vs. CL

'''
To prevent Colab from time out, go to the google Colab console (ctrl+shift+i) and type:
function ClickConnect(){console.log("Working");document.querySelector("colab-toolbar-button#connect").click()}setInterval(ClickConnect,60000)

Don't exit the console until you get "Working" as the output in the console window. 
It would keep on clicking the page and prevent it from disconnecting.
Make sure you dont run anything for more than 12 hrs on Colab!
'''
##########################################
import numpy
from numpy.random import seed
seed(2)
!pip install tensorflow
import tensorflow
tensorflow.random.set_seed(2)
# set_random_seed(2)


############################################
# import the required modules
from __future__ import print_function
import pandas as pd
import numpy as np
import csv
import sys
import time

import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import *
from keras.callbacks import EarlyStopping
#from keras.optimizers import Adamax
from tensorflow.keras.optimizers import Adamax

from sklearn.metrics import confusion_matrix,f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize,LabelEncoder
from sklearn.utils import class_weight

# Attention implemented by https://github.com/CyberZHG/keras-self-attention
!pip install keras-self-attention 
from keras_self_attention import SeqSelfAttention

# experiment options
# windows = [4, 6, 8]
windows = [4]
modes = ['switch']


# number of features for switch / ps / as tests
nb_switch = 8
nb_ps = 1
nb_as = 1
nb_multi = nb_switch + nb_ps + nb_as      #######

# lstm_sizes = [[8,4,2],[16,8,4],[32,16,8],[128,64,32],[256,128,64],[512,256,128],[1024,512,256]]


# BLSTM variables
# lstm_sizes = [[8,4,2],[16,8,4],[32,16,8],[128,64,32]] # number of neurons in the BLSTM layers
time_steps = [1]  # input history to include
attention_widths = [1] # width of the local context for the attention layer
lstm_sizes = [[128,64,32]]
# other parameters
batch_size = 32 # for estimating error gradient
nb_class = 2 # number of classes
nb_epoch = 50 # number of total epochs to train the model

# optimization function
opt_func = Adamax(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08) 
# to prevent over-fitting
early_stopping = EarlyStopping(monitor='loss', patience=20)

# access data from Google Drive
from google.colab import drive
drive.mount('/content/drive')
# dir_log = '/content/drive/My Drive/FIT5126-Minor-Thesis/outputs/exp3/binary_'
dir_data = '/content/drive/My Drive/FIT5126-Minor-Thesis/data_Faezeh/processed/window_0nan/'
# printout = '/content/drive/My Drive/FIT5126-Minor-Thesis/outputs/logs/binary_printout_'





# turn off the warnings, be careful when use this
import warnings
warnings.filterwarnings("ignore")

# # save outputs to a log file
# ts = time.time()
# printout_f = printout + str(ts) + '.txt'
# idlestdout = sys.stdout
# logger = open(printout_f, "w")
# sys.stdout = logger

# reshape panda.DataFrame to Keras style: (batch_size, time_step, nb_features)
def reshape_data(data, n_prev):
    docX = []
    for i in range(len(data)):
        if i < (len(data)-n_prev):
            docX.append(data.iloc[i:i+n_prev].values)
        else: # the frames in the last window use the same context
            docX.append(data.iloc[(len(data)-n_prev):len(data)].values)
    alsX = np.array(docX)
    return alsX

# define the BLSTM model with attention
def attBLSTM(lstm_size, attention_width, nb_class, opt_func):
    model = Sequential()
    model.add(Bidirectional(LSTM(units=lstm_size[0], return_sequences=True))) # BLSTM layer 1
    model.add(Bidirectional(LSTM(units=lstm_size[1], return_sequences=True))) # BLSTM layer 2
    model.add(Bidirectional(LSTM(units=lstm_size[2], return_sequences=True))) # BLSTM layer 3
    # model.add(Bidirectional(LSTM(units=lstm_size[3], return_sequences=True))) # BLSTM layer 4
    # model.add(Bidirectional(LSTM(units=lstm_size[4], return_sequences=True))) # BLSTM layer 5
    model.add(SeqSelfAttention(attention_width=attention_width, attention_activation='sigmoid')) # attention layer
    model.add(Dense(units=nb_class, activation='softmax')) # output layer, predict emotion dimensions seperately
    return model

# experiments
count = 1
for window in windows:       
# for dataset in sets:
    for mode in modes: 
        # data files
        # file_log = dir_log + 'aln_' + 'window_' + str(window) + '_' + mode + '_log.txt'   #########
        file_data_train = dir_data + str(window) + '/combined_train_'  + mode + '.csv'        #########
        file_data_dev = dir_data + str(window) + '/combined_dev_'  + mode + '.csv'          #########
        file_data_test = dir_data + str(window) + '/combined_test_' + mode + '.csv'         #########

        nb_features = globals()[str('nb_' + mode)] * window

        
        # read in data
        data_train = pd.read_csv(file_data_train, usecols=range(nb_features), na_values = ' ', header=None)            #########
        data_dev = pd.read_csv(file_data_dev, usecols=range(nb_features), na_values = ' ', header=None)                #########
        data_test = pd.read_csv(file_data_test, usecols=range(nb_features), na_values = ' ', header=None)              #########
        
        label_data_train = pd.read_csv(file_data_train, usecols=[(nb_features)], header=None)                        #########
        label_raw_data_train = label_data_train.values                                                                  #########
        
        label_data_dev = pd.read_csv(file_data_dev , usecols=[(nb_features)], header=None)                            #########
        label_raw_data_dev  = label_data_dev .values                                                                    #########
        
        label_data_test = pd.read_csv(file_data_test , usecols=[(nb_features)], header=None)                           #########
        label_raw_data_test  = label_data_test .values                                                                   #########
        
        # one-hot encoding of the classes for train dataset
        label_conv_train = []
        for item in label_raw_data_train:
            if item == 'CL':
                converted_label = [1,0] # CL
            else:
                converted_label = [0,1] # VS or VM
            label_conv_train.append(converted_label)
        label_conv_train = np.asarray(label_conv_train)
        label_conv_train_df = pd.DataFrame(label_conv_train, index=None)

        # one-hot encoding of the classes for development dataset
        label_conv_dev = []
        for item in label_raw_data_dev:
            if item == 'CL':
                converted_label = [1,0] # CL
            else:
                converted_label = [0,1] # VS or VM
            label_conv_dev.append(converted_label)
        label_conv_dev = np.asarray(label_conv_dev)
        label_conv_dev_df = pd.DataFrame(label_conv_dev, index=None)


        # one-hot encoding of the classes for test dataset
        label_conv_test = []
        for item in label_raw_data_test:
            if item == 'CL':
                converted_label = [1,0] # CL
            else:
                converted_label = [0,1] # VS or VM
            label_conv_test.append(converted_label)
        label_conv_test = np.asarray(label_conv_test)
        label_conv_test_df = pd.DataFrame(label_conv_test, index=None)


        # Grid search for best parameters
        para_list = []
        tst_pred_list = []
        for time_step in time_steps:
            X_train = reshape_data(data_train, time_step) # pad feature data
            y_train = reshape_data(label_conv_train_df, time_step) # pad label data

            X_dev = reshape_data(data_dev, time_step) # pad feature data
            y_dev = reshape_data(label_conv_dev_df, time_step) # pad label data

            X_test = reshape_data(data_test, time_step) # pad feature data
            y_test = reshape_data(label_conv_test_df, time_step) # pad label data

            for lstm_size in lstm_sizes:
                for attention_width in attention_widths:
                    para_list.append([time_step, lstm_size, attention_width]) # save parameter set 
                    print('\n================================ No. %s of 16 ========================================' % count)                   
                    print('\n Tranining on:%s ,dataset\n' % (mode) )
                    # build model with given parameters
                    model = attBLSTM(lstm_size, attention_width, nb_class, opt_func)
                    model.compile(loss='categorical_crossentropy', optimizer=opt_func, metrics=['categorical_accuracy'])
                    # training the model
                    #x = np.asarray(x).astype('float32')
                    model.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epoch, validation_data=(X_dev, y_dev), verbose=0)
                    # evaluation
                    model.evaluate(X_test, y_test, batch_size=batch_size)

                    # print confusion matrix
                    y_test_non_category = [ np.argmax(t[0]) for t in y_test ]
                    tst_pred = model.predict(X_test)
                    y_predict_non_category = [ np.argmax(t[0]) for t in tst_pred ]

                    tst_f1 = f1_score(y_test_non_category, y_predict_non_category, average='weighted')
                    print('\nParameters: mode = %s, time_step = %s, [h1, h2, h3] = %s, attention_width = %s\n' % (mode, time_step, lstm_size, attention_width))
                    print('Confusion Matrix on test set')
                    print(confusion_matrix(y_test_non_category, y_predict_non_category))
                    print('Weighted F1-score on test set:', tst_f1)
                    count += 1



In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
!pip install lime
import lime
from lime import lime_tabular

%matplotlib inline

# Attention implemented by https://github.com/CyberZHG/keras-self-attention
!pip install keras-self-attention 
from keras_self_attention import SeqSelfAttention

#eli5 package (https://eli5.readthedocs.io/en/latest)
!pip install eli5
import eli5
from eli5.sklearn import PermutationImportance

#lime package (https://github.com/marcotcr/lime)
import lime
import lime.lime_tabular

In [ ]:
def PredFunc (XTest):
  tst_pred = model.predict(XTest)

  ff = [ t[0] for t in tst_pred ]
  
  list_ff = []
  for item in ff:
    item = item.tolist()
    list_ff.append(item)
  
  ff_aray = np.array(list_ff)
  ff_aray
  
  return(ff_aray)

In [ ]:
header = ['1', '2','3','4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32']
random_state=123

explainer = lime_tabular.RecurrentTabularExplainer(X_train, training_labels=y_train, feature_names=header, random_state=random_state,
                                                   discretize_continuous=True,
                                                   class_names=['Healthy', 'Patient'],
                                                   discretizer='decile')

In [ ]:
# print(X_test[0])
print(y_test[0])

print('\n\n--------------------\n\n')

y_predict = np.argmax(model.predict(X_test), axis=-1)
print(y_predict[0])

print('\n\n--------------------\n\n')

predictions = (model.predict(X_test) < 0.5).astype("int32")
predictions[0]

In [ ]:
# SWITCH-VS-VS

exp = explainer.explain_instance(X_test[372], PredFunc, num_features=10, labels=(1,))
exp.show_in_notebook(show_table=True)
plt = exp.as_pyplot_figure()
plt.tight_layout()

In [ ]:
# SWITCH-VS-CL

exp = explainer.explain_instance(X_test[837], PredFunc, num_features=10, labels=(1,))
exp.show_in_notebook(show_table=True)
plt = exp.as_pyplot_figure()
plt.tight_layout()

In [ ]:
# SWITCH-CL-CL

exp = explainer.explain_instance(X_test[93], PredFunc, num_features=10, labels=(1,))
exp.show_in_notebook(show_table=True)
plt = exp.as_pyplot_figure()
plt.tight_layout()

In [ ]:
# SWITCH-CL-VS

exp = explainer.explain_instance(X_test[0], PredFunc, num_features=10, labels=(1,))
exp.show_in_notebook(show_table=True)
plt = exp.as_pyplot_figure()
plt.tight_layout()